In [1]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings('ignore')

['preprocessing']


In [2]:
#importing the training data
imdb_data=pd.read_csv('/kaggle/input/preprocessing/train.csv')
imdb_data_test=pd.read_csv('/kaggle/input/preprocessing/test.csv')
print(imdb_data.shape)
imdb_data.head(10)

(30000, 2)


,review,sentiment
0,SAPS AT SEA <br /><br />Aspect ratio: 1.37:1<b...,negative
1,"If you want mindless action, hot chicks and a ...",positive
2,"""The Woman in Black"" is easily one of the cree...",positive
3,I can barely find the words to describe how mu...,negative
4,What's in here ?! Let me tell you. It's the pr...,negative
5,"This is the story of a maniac cop who, for som...",negative
6,Before I continue forth with the new millenniu...,positive
7,"When Rodney Dangerfield is on a roll, he's hil...",negative
8,Prom Night is shot with the artistic eye someo...,negative
9,"""Destroy All Planets"" winds up settling for 'd...",negative


In [36]:
def preprocess_text(text, method='stemming'):
    # Define the tokenizer
    tokenizer = ToktokTokenizer()
    
    # Define stopword list
    stopword_list = nltk.corpus.stopwords.words('english')
    
    # Removing HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    
    # Removing text inside square brackets
    text = re.sub('\[[^]]*\]', '', text)
    
    # Define function for removing special characters
    def remove_special_characters(text, remove_digits=True):
        pattern = r'[^a-zA-z0-9\s]'
        text = re.sub(pattern, '', text)
        return text
    
    # Remove special characters
    text = remove_special_characters(text)
    
    # Tokenization
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    # Stemming or Lemmatization
    if method == 'stemming':
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    elif method == 'lemmatization':
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Removing the stopwords
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    
    # Reconstruct the text
    text = ' '.join(filtered_tokens)
    
    return text

# Example usage
method = 'lemmatization' # or 'stemming'
imdb_data['review'] = imdb_data['review'].apply(lambda x: preprocess_text(x, method=method))
imdb_data_test['review'] = imdb_data_test['review'].apply(lambda x: preprocess_text(x, method=method))


In [37]:
#Count vectorizer for bag of words
cv=CountVectorizer(max_features = 60000, ngram_range = (1,3), max_df = 0.8, min_df = 3)
#transformed train reviews
cv_train_reviews=cv.fit_transform(imdb_data['review'])
cv_test_reviews=cv.transform(imdb_data_test['review'])


print('BOW_cv_train:',cv_train_reviews.shape)




print('BOW_cv_test:',cv_test_reviews.shape)

#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (30000, 60000)
BOW_cv_test: (20000, 60000)


In [30]:
tv=TfidfVectorizer(max_features = 50000, ngram_range = (1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(imdb_data['review'])
tv_test_reviews=tv.transform(imdb_data_test['review'])


print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)


Tfidf_train: (30000, 50000)
Tfidf_test: (20000, 50000)


In [38]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
sentiment_data_test=lb.transform(imdb_data_test['sentiment'])
print(sentiment_data.shape)
print(sentiment_data_test.shape)

(30000, 1)
(20000, 1)


In [39]:
#Spliting the sentiment data
sentiment_data=sentiment_data[:30000]
sentiment_data_test=sentiment_data_test[:20000]

print(sentiment_data)
print(sentiment_data_test)


[[0]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]
[[1]
 [1]
 [0]
 ...
 [0]
 [1]
 [0]]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import time

# Initialize the KNN model
knn = KNeighborsClassifier()

# Start timing
start_time = time.time()

# Fitting the KNN model for bag of words
knn_bow = knn.fit(tv_train_reviews, sentiment_data)
print(knn_bow)

# Model prediction
knn_bow_predict = knn.predict(tv_test_reviews)

# End timing
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(knn_bow_predict)
print(f"Time taken for model to run: {elapsed_time} seconds")

# Accuracy score for bag of words
knn_bow_score = accuracy_score(sentiment_data_test, knn_bow_predict)
print("knn_bow_score :", knn_bow_score)


In [48]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import time

# Initialize the XGBoost classifier model
xgb = XGBClassifier(
    n_estimators=1000,    # Start with a moderate number and increase if needed
    learning_rate=0.1,   # Lower learning rate for a start
    max_depth=7,         # Start with a moderate depth
    min_child_weight=1,  # Default value
    gamma=0.1,           # Slight regularization
    subsample=0.8,       # 80% of data to grow trees prevents overfitting
    colsample_bytree=0.8, # 80% of features used
    objective='binary:logistic', # Since the dataset is balanced and binary classification
    use_label_encoder=False, # To avoid warning since XGBClassifier uses a label encoder internally by default
    eval_metric='logloss' # Evaluation metric for binary classification
)

# Fitting the XGBoost model for bag of words
print("Training XGBoost model...")
start_train_time = time.time()
xgb_bow = xgb.fit(cv_train_reviews, sentiment_data)
end_train_time = time.time()
print("Training completed.")

# Start timing for prediction
start_predict_time = time.time()

# Model prediction
xgb_bow_predict = xgb.predict(cv_test_reviews)

# End timing for prediction
end_predict_time = time.time()

# Calculate the elapsed time
train_elapsed_time = end_train_time - start_train_time
predict_elapsed_time = end_predict_time - start_predict_time

print(xgb_bow_predict)
print(f"Time taken to train the model: {train_elapsed_time} seconds")
print(f"Time taken for model to predict: {predict_elapsed_time} seconds")

# Accuracy score for bag of words
xgb_bow_score = accuracy_score(sentiment_data_test, xgb_bow_predict)
print("xgb_bow_score:", xgb_bow_score)

Training XGBoost model...
Training completed.
[1 1 0 ... 0 1 0]
Time taken to train the model: 97.90205764770508 seconds
Time taken for model to predict: 0.9559218883514404 seconds
xgb_bow_score: 0.88725


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import time

# Initialize the Multinomial Naive Bayes model
mnb = MultinomialNB()

# Fitting the MNB model for bag of words
mnb_bow = mnb.fit(tv_train_reviews, sentiment_data)
print(mnb_bow)

# Start timing
start_time = time.time()

# Model prediction
mnb_bow_predict = mnb.predict(tv_test_reviews)

# End timing
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(mnb_bow_predict)
print(f"Time taken for model to run: {elapsed_time} seconds")

# Accuracy score for bag of words
mnb_bow_score = accuracy_score(sentiment_data_test, mnb_bow_predict)
print("mnb_bow_score :", mnb_bow_score)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

# Initialize the Random Forest model
rf = RandomForestClassifier()

# Start timing
start_time = time.time()

# Fitting the RF model for bag of words
rf_bow = rf.fit(tv_train_reviews, sentiment_data)
print(rf_bow)

# Model prediction
rf_bow_predict = rf.predict(tv_test_reviews)

# End timing
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(rf_bow_predict)
print(f"Time taken for model to run: {elapsed_time} seconds")

# Accuracy score for bag of words
rf_bow_score = accuracy_score(sentiment_data_test, rf_bow_predict)
print("rf_bow_score :", rf_bow_score)